In [1]:
# problema de classificação multiclasses
# problema de dimensionalidade

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

Leitura dos arquivos

In [2]:
train = pd.read_csv('data/train.csv').sample(5000)
#train = pd.read_csv('data/train.csv')
print('train: ' + str(train.shape))
test = pd.read_csv('data/test.csv')
print('test: ' + str(test.shape))
train.head()

train: (5000, 785)
test: (28000, 784)


,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
19062,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28931,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16838,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15931,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12938,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
print(train.shape)

(5000, 785)


In [4]:
# feature matrix
X = train.iloc[:,1:] # todas as colunas tirando label
# response vector
y = train['label'] # nosso target variavel que iremos prever

Este problema é típico da para trabalhar a dimensionalidade do dataset, para isso será aplicado PCA.

In [5]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)  


PCA é melhor com um conjunto de recursos normalizados. Vou realizar a normalização escalar padrão para normalizar nosso conjunto de recursos.

In [6]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()  
X_train = sc.fit_transform(X_train)  
test   = sc.transform(test)

In [11]:
from sklearn.decomposition import PCA

pca = PCA(.86)  
X_train = pca.fit_transform(X_train)  
test   = pca.transform(test)

In [12]:
explained_variance = pca.explained_variance_ratio_  

#trasnforma em pandas dataframe para contar quantos atributos são necessários para obter 0.98 de variancia
t = pd.Series(explained_variance).count()

#print(explained_variance)
print(t)

105


In [13]:
from sklearn.decomposition import PCA

pca = PCA(n_components=t)  
X_train = pca.fit_transform(X_train)  
test   = pca.transform(test)


In [14]:
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV

print(pd.Timestamp.now())
        
parameters = {'kernel':('linear', 'rbf'), 'C':[0.1, 1, 5, 50, 100, 150], 'gamma': [0.0001, 0.0005, 0.001, 0.005,1 ,2,5 ,10, 100]}
svc = svm.SVC()
clf = GridSearchCV(svc, parameters, cv = 5)
clf.fit(X_train, y_train) 

print(clf.best_params_)
print(clf.best_score_)

print(pd.Timestamp.now())

#{'kernel': 'rbf', 'C': 5, 'gamma': 0.0005} --0.925

2018-07-25 10:11:34.030000
{'kernel': 'rbf', 'C': 100, 'gamma': 0.001}
0.9334285714285714
2018-07-25 10:24:26.852000


In [15]:

predictions = clf.predict(test)

result = pd.DataFrame({'ImageId': list(range(1,len(predictions)+1)), 'Label': predictions})
result.to_csv('result.csv', index=False, header=True)